In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np


In [ ]:
max_frames = 20
X_train = [
    np.random.random((15, 100, 100, 1)),  # Video 1 con 15 frames
    np.random.random((20, 100, 100, 1)),  # Video 2 con 20 frames
    np.random.random((12, 100, 100, 1)),  # Video 3 con 12 frames
]
for i, video in enumerate(X_train):
    frames_actual = video.shape[0]
    if frames_actual < max_frames:
        
        padding = np.zeros((max_frames - frames_actual, 100, 100, 1))
        X_train[i] = np.concatenate([video, padding], axis=0)
    elif frames_actual > max_frames:
        # Truncar si es necesario
        X_train[i] = video[:max_frames, :, :, :]
        
X_train = np.array(X_train)
y_train = np.array([1, 0, 1])  # Etiquetas de ejemplo

In [ ]:

model = models.Sequential()
model.add(layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu'), input_shape=(max_frames, 100, 100, 1)))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.MaxPooling2D((2, 2))))
model.add(layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')))
model.add(layers.TimeDistributed(layers.Flatten()))
model.add(layers.LSTM(64))
model.add(layers.Dense(1, activation='sigmoid'))


In [ ]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)


In [ ]:

nuevo_video = np.random.random((18, 100, 100, 1))

max_frames = 20
if nuevo_video.shape[0] < max_frames:
    # Rellenar con ceros si es necesario
    padding = np.zeros((max_frames - nuevo_video.shape[0], 100, 100, 1))
    nuevo_video = np.concatenate([nuevo_video, padding], axis=0)
elif nuevo_video.shape[0] > max_frames:
    # Truncar si es necesario
    nuevo_video = nuevo_video[:max_frames, :, :, :]

# Realizar la predicción
prediccion = model.predict(np.expand_dims(nuevo_video, axis=0))

print(f'Predicción para el nuevo video: {prediccion[0][0]}')